# Predict VoxCeleb utterance info from SSL representations

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.chdir('../..')
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

In [ ]:
from notebooks.notebooks_utils import (
    load_models,
    evaluate_models,
    create_metrics_df
)

from sslsv.evaluations.CosineSVEvaluation import CosineSVEvaluation, CosineSVEvaluationTaskConfig

In [ ]:
models = load_models(
    [
        './models/old/vox2_ddp_sntxent_s=30_m=0/config.yml',
        './models/old/vox2_ddp_sntxent_s=30_m=0.1/config.yml'
    ],
    override_names={
        'models/old/vox2_ddp_sntxent_s=30_m=0'   : 'simclr',
        'models/old/vox2_ddp_sntxent_s=30_m=0.1' : 'simclr_am'
    }
)

In [ ]:
evaluate_models(models, CosineSVEvaluation, CosineSVEvaluationTaskConfig())

In [ ]:
from sklearn.linear_model import LogisticRegression

def fit_mlp_on_representations(model, y_key_pos):
    keys = list(model['embeddings'].keys())
    
    X = [model['embeddings'][key][0].numpy() for key in keys]
    if y_key_pos is None:
        y = keys
    else:
        y = [key.split('/')[y_key_pos] for key in keys]
    
    clf = LogisticRegression()
    clf.fit(X, y)
    
    print(f'Accuracy: {clf.score(X, y)}')
    
    return clf, X, y

## Speaker ID

In [ ]:
_ = fit_mlp_on_representations(models['simclr'], y_key_pos=1)

In [ ]:
_ = fit_mlp_on_representations(models['simclr_am'], y_key_pos=1)

## Video ID

In [ ]:
_ = fit_mlp_on_representations(models['simclr'], y_key_pos=2)

In [ ]:
_ = fit_mlp_on_representations(models['simclr_am'], y_key_pos=2)

## Segment ID

In [ ]:
_ = fit_mlp_on_representations(models['simclr'], y_key_pos=3)

In [ ]:
_ = fit_mlp_on_representations(models['simclr_am'], y_key_pos=3)

## Sample ID

In [ ]:
_ = fit_mlp_on_representations(models['simclr'], y_key_pos=None)

In [ ]:
_ = fit_mlp_on_representations(models['simclr_am'], y_key_pos=None)